<a href="https://colab.research.google.com/github/FatemehNMT/MyROS/blob/main/2_DOF_Arm_Robot_and_Sensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2-DOF Arm Robot and Sensors:

1. Generate a based moving robot (base_link).
2. Set a 2-DOF arm on it(arm_joint1 and arm_joint2).
3. Put a sensor (camera or laser) at the end of the arm (camera_link).
4. Broadcast tf (tf broadcaster)
5. Define  and send out different frames (base_link, arm_joint1, arm_joint2, camera_link, object_frame).
6. Represent tf-tree applying rqt_tf_tree.
7. Apply tf listener to measure the distance of object's location relevant to the base or the camera.
8. Apply lookup_transform.
9. Present all the results in RViz with axes and markers.

Project Structure:

my_tf_demo/
├── urdf/
│   └── robot_with_arm.urdf.xacro
├── launch/
│   ├── display.launch
│   ├── gazebo.launch
│   └── tf_demo.launch
├── scripts/
│   ├── tf_broadcaster.py
│   └── tf_listener.py
└── CMakeLists.txt
└── package.xml

In [ ]:
cd simulation_ws/src/
catkin_create_pkg my_tf_demo roscpp rospy urdf sensor_msgs std_msgs
cd ..
catkin_make
source devel/setup.bash

In [ ]:
# if we are located in the simulation_ws folder
chmod +x src/my_tf_demo/scripts/tf_broadcaster.py
chmod +x src/my_tf_demo/scripts/tf_listener.py
chmod +x src/my_tf_demo/scripts/gazebo_object_tf.py

In [ ]:
catkin_make

## **Step 1**

urdf/robot_with_arm.urdf.xacro

In [ ]:
<?xml version="1.0"?>
<robot name="tf_demo_robot" xmlns:xacro="http://ros.org/wiki/xacro">

  <!-- Base link -->
  <link name="base_link">
    <visual>
      <geometry><box size="0.4 0.4 0.2"/></geometry>
      <material name="red"><color rgba="0.8 0.2 0.2 1"/></material>
    </visual>
    <collision>
      <geometry><box size="0.4 0.4 0.2"/></geometry>
    </collision>
    <inertial>
      <mass value="5"/>
      <inertia ixx="0.1" ixy="0" ixz="0" iyy="0.1" iyz="0" izz="0.1"/>
    </inertial>
  </link>

  <!-- First joint + link -->
  <joint name="arm_joint1" type="revolute">
    <parent link="base_link"/>
    <child link="arm_link1"/>
    <origin xyz="0 0 0.15" rpy="0 0 0"/>
    <axis xyz="0 0 1"/>
    <limit effort="10" velocity="1.0" lower="-1.57" upper="1.57"/>
  </joint>

  <link name="arm_link1">
    <visual>
      <geometry><cylinder radius="0.03" length="0.4"/></geometry>
      <origin xyz="0 0 0.2" rpy="0 0 0"/>
      <material name="yellow"><color rgba="0.9 0.9 0.2 1"/></material>
    </visual>
    <collision>
      <geometry><cylinder radius="0.03" length="0.4"/></geometry>
    </collision>
    <inertial>
      <mass value="0.5"/>
      <inertia ixx="0.01" ixy="0" ixz="0" iyy="0.01" iyz="0" izz="0.01"/>
    </inertial>
  </link>

  <!-- Second joint + link -->
  <joint name="arm_joint2" type="revolute">
    <parent link="arm_link1"/>
    <child link="arm_link2"/>
    <origin xyz="0 0 0.4" rpy="0 0 0"/>
    <axis xyz="0 1 0"/>
    <limit effort="10" velocity="1.0" lower="-1.57" upper="1.57"/>
  </joint>

  <link name="arm_link2">
    <visual>
      <geometry><cylinder radius="0.025" length="0.3"/></geometry>
      <origin xyz="0 0 0.15" rpy="0 0 0"/>
      <material name="blue"><color rgba="0.3 0.3 0.8 1"/></material>
    </visual>
    <collision>
      <geometry><cylinder radius="0.025" length="0.3"/></geometry>
    </collision>
    <inertial>
      <mass value="0.3"/>
      <inertia ixx="0.005" ixy="0" ixz="0" iyy="0.005" iyz="0" izz="0.005"/>
    </inertial>
  </link>

  <!-- Camera link -->
  <joint name="camera_joint" type="fixed">
    <parent link="arm_link2"/>
    <child link="camera_link"/>
    <origin xyz="0 0 0.3" rpy="0 0 0"/>
  </joint>

  <link name="camera_link">
    <visual>
      <geometry><box size="0.1 0.05 0.05"/></geometry>
      <material name="green"><color rgba="0.2 0.8 0.2 1"/></material>
    </visual>
    <collision>
      <geometry><box size="0.1 0.05 0.05"/></geometry>
    </collision>
    <inertial>
      <mass value="0.1"/>
      <inertia ixx="0.001" ixy="0" ixz="0" iyy="0.001" iyz="0" izz="0.001"/>
    </inertial>
  </link>

</robot>


launch/display.launch

In [ ]:
<launch>
  <!-- Load URDF -->
  <param name="robot_description" command="$(find xacro)/xacro $(find my_tf_demo)/urdf/robot_with_arm.urdf.xacro" />

  <!-- Joint State Publisher GUI -->
  <node name="joint_state_publisher_gui" pkg="joint_state_publisher_gui" type="joint_state_publisher_gui" />

  <!-- Robot State Publisher -->
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher" />

  <!-- Map -->
  <node pkg="tf2_ros" type="static_transform_publisher" name="map_to_base"
      args="0 0 0 0 0 0 map base_link" />

  <!-- RViz -->
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find my_tf_demo)/urdf/display.rviz" required="true"/>
</launch>


In [ ]:
# Run
roslaunch my_tf_demo display.launch

To see tree of relationships

In [ ]:
# used to visualize the relationships between coordinate frames being broadcast
# by the tf (or tf2 in newer ROS versions) system.
rosrun tf view_frames
# Will save a PDF file of this relationship in the current folder.

In [ ]:
# Or:
rosrun rqt_tf_tree rqt_tf_tree
# Savev Image

gazebo.launch

## **Step 2**

Add an objet to the scene.
And broadcast its location on TF.

scripts/tf_broadcaster.py

In [ ]:
#!/usr/bin/env python3
import rospy
import tf_conversions
import tf2_ros
import geometry_msgs.msg
import math

def broadcaster():
    rospy.init_node('object_tf_broadcaster')

    br = tf2_ros.TransformBroadcaster()
    rate = rospy.Rate(10)

    while not rospy.is_shutdown():
        t = geometry_msgs.msg.TransformStamped()

        t.header.stamp = rospy.Time.now()
        t.header.frame_id = "base_link"
        t.child_frame_id = "object_frame"

        # Object position relative to base_link
        t.transform.translation.x = 1.0
        t.transform.translation.y = 0.5
        t.transform.translation.z = 0.0

        # No rotation (identity quaternion)
        q = tf_conversions.transformations.quaternion_from_euler(0, 0, 0)
        t.transform.rotation.x = q[0]
        t.transform.rotation.y = q[1]
        t.transform.rotation.z = q[2]
        t.transform.rotation.w = q[3]

        br.sendTransform(t)
        rate.sleep()

if __name__ == '__main__':
    try:
        broadcaster()
    except rospy.ROSInterruptException:
        pass


This node prints the distance and angle of the object relative to the camera:

scripts/tf_listener.py

In [ ]:
#!/usr/bin/env python3
import rospy
import tf2_ros
import geometry_msgs.msg
import math

def listener():
    rospy.init_node('object_tf_listener')

    tfBuffer = tf2_ros.Buffer()
    listener = tf2_ros.TransformListener(tfBuffer)

    rate = rospy.Rate(2)
    while not rospy.is_shutdown():
        try:
            # Transform from camera_link to object_frame
            trans = tfBuffer.lookup_transform("camera_link", "object_frame", rospy.Time(0))

            x = trans.transform.translation.x
            y = trans.transform.translation.y
            z = trans.transform.translation.z

            dist = math.sqrt(x**2 + y**2 + z**2)
            angle = math.atan2(y, x)

            rospy.loginfo("Object wrt camera_link: (x=%.2f, y=%.2f, z=%.2f), dist=%.2f m, angle=%.2f rad",
                          x, y, z, dist, angle)

        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
            continue

        rate.sleep()

if __name__ == '__main__':
    listener()


launch/tf_demo.launch

In [ ]:
<launch>
  <!-- Load URDF -->
  <param name="robot_description" command="$(find xacro)/xacro $(find my_tf_demo)/urdf/robot_with_arm.urdf.xacro" />

  <!-- State publisher -->
  <node name="joint_state_publisher_gui" pkg="joint_state_publisher_gui" type="joint_state_publisher_gui"/>
  <node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher"/>

  <!-- RViz -->
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find my_tf_demo)/urdf/display.rviz" />

  <!-- Object broadcaster -->
  <node name="tf_broadcaster" pkg="my_tf_demo" type="tf_broadcaster.py" output="screen"/>

  <!-- Listener -->
  <node name="tf_listener" pkg="my_tf_demo" type="tf_listener.py" output="screen"/>
</launch>


In [ ]:
roslaunch my_tf_demo tf_demo.launch

## **Add the object to Gazebo**

urdf/object.sdf

In [ ]:
<?xml version="1.0" ?>
<sdf version="1.6">
  <model name="object_box">
    <static>true</static>
    <link name="object_link">
      <pose>1 0.5 0 0 0 0</pose> <!-- x=1.0, y=0.5, z=0.0 -->
      <visual name="visual">
        <geometry>
          <box>
            <size>0.2 0.2 0.2</size>
          </box>
        </geometry>
        <material>
          <ambient>0 0.8 0.8 1</ambient>
          <diffuse>0 0.8 0.8 1</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <geometry>
          <box>
            <size>0.2 0.2 0.2</size>
          </box>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>


Gazebo + robot + object

launch/gazebo.launch

In [ ]:
<launch>
  <!-- Start Gazebo with empty world -->
  <include file="$(find gazebo_ros)/launch/empty_world.launch" />

  <!-- Load robot description -->
  <param name="robot_description" command="$(find xacro)/xacro $(find my_tf_demo)/urdf/robot_with_arm.urdf.xacro" />

  <!-- Spawn robot in Gazebo -->
  <node name="spawn_urdf" pkg="gazebo_ros" type="spawn_model"
        args="-param robot_description -urdf -model tf_demo_robot -x 0 -y 0 -z 0.5" />

  <!-- Spawn object -->
  <node name="spawn_object" pkg="gazebo_ros" type="spawn_model"
        args="-file $(find my_tf_demo)/urdf/object.sdf -sdf -model object_box -x 1 -y 0.5 -z 0.1" />
</launch>


Synchronizing tf with object model

When you spawn a model, Gazebo creates a link to it. This link can be found in /gazebo/link_states.
We can publish a tf object from this data with a simple node.

scripts/gazebo_object_tf.py

In [ ]:
#!/usr/bin/env python3
import rospy
import tf_conversions
import tf2_ros
import geometry_msgs.msg
from gazebo_msgs.msg import LinkStates

def callback(data):
    if "object_box::object_link" in data.name:
        idx = data.name.index("object_box::object_link")
        pose = data.pose[idx]

        br = tf2_ros.TransformBroadcaster()
        t = geometry_msgs.msg.TransformStamped()

        t.header.stamp = rospy.Time.now()
        t.header.frame_id = "world"
        t.child_frame_id = "object_frame"

        t.transform.translation.x = pose.position.x
        t.transform.translation.y = pose.position.y
        t.transform.translation.z = pose.position.z

        t.transform.rotation = pose.orientation

        br.sendTransform(t)

def listener():
    rospy.init_node('gazebo_object_tf')
    rospy.Subscriber("/gazebo/link_states", LinkStates, callback)
    rospy.spin()

if __name__ == '__main__':
    listener()


Final Launch

launch/tf_demo_gazebo.launch

In [ ]:
<launch>
  <!-- Start Gazebo with empty world -->
  <include file="$(find gazebo_ros)/launch/empty_world.launch" />

  <!-- Load robot description -->
  <!-- <param name="robot_description" command="$(find xacro)/xacro $(find my_tf_demo)/urdf/robot_with_arm.urdf.xacro" /> -->
  <param name="robot_description" command="$(find xacro)/xacro --inorder $(find my_tf_demo)/urdf/robot_with_arm.urdf.xacro" />
  <!-- or -->
  <!-- <param name="robot_description" command="$(xacro $(find my_tf_demo)/urdf/robot_with_arm.urdf.xacro)" /> -->

  <!-- Spawn robot in Gazebo -->
  <node name="spawn_urdf" pkg="gazebo_ros" type="spawn_model"
        args="-param robot_description -urdf -model tf_demo_robot -x 0 -y 0 -z 0.5" />

  <!-- Spawn object -->
  <node name="spawn_object" pkg="gazebo_ros" type="spawn_model"
        args="-file $(find my_tf_demo)/urdf/object.sdf -sdf -model object_box -x 1 -y 0.5 -z 0.1" />
</launch>

In [ ]:
roslaunch my_tf_demo tf_demo_gazebo.launch

It will be seen that, there is no movement in the Gazebo. It happens since, there is no joint controller or transmission defined for the robot.

urdf (robot_with_arm.urdf.xacro)
Add gazebo plugin and transmission to each joint.

In [ ]:
<?xml version="1.0"?>
<robot name="tf_demo_robot" xmlns:xacro="http://ros.org/wiki/xacro">

  <!-- Base link -->
  <link name="base_link">
    <visual>
      <geometry><box size="0.4 0.4 0.2"/></geometry>
      <material name="red"><color rgba="0.8 0.2 0.2 1"/></material>
    </visual>
    <collision>
      <geometry><box size="0.4 0.4 0.2"/></geometry>
    </collision>
    <inertial>
      <mass value="5"/>
      <inertia ixx="0.1" ixy="0" ixz="0" iyy="0.1" iyz="0" izz="0.1"/>
    </inertial>
  </link>

  <!-- First joint + link -->
  <joint name="arm_joint1" type="revolute">
    <parent link="base_link"/>
    <child link="arm_link1"/>
    <origin xyz="0 0 0.15" rpy="0 0 0"/>
    <axis xyz="0 0 1"/>
    <limit effort="10" velocity="1.0" lower="-1.57" upper="1.57"/>
  </joint>

  <link name="arm_link1">
    <visual>
      <geometry><cylinder radius="0.03" length="0.4"/></geometry>
      <origin xyz="0 0 0.2" rpy="0 0 0"/>
      <material name="yellow"><color rgba="0.8 0.8 0.2 1"/></material>
    </visual>
    <collision>
      <geometry><cylinder radius="0.03" length="0.4"/></geometry>
    </collision>
    <inertial>
      <mass value="1"/>
      <inertia ixx="0.01" ixy="0" ixz="0" iyy="0.01" iyz="0" izz="0.01"/>
    </inertial>
  </link>

  <!-- Second joint + link -->
  <joint name="arm_joint2" type="revolute">
    <parent link="arm_link1"/>
    <child link="arm_link2"/>
    <origin xyz="0 0 0.4" rpy="0 0 0"/>
    <axis xyz="0 1 0"/>
    <limit effort="10" velocity="1.0" lower="-1.57" upper="1.57"/>
  </joint>

  <link name="arm_link2">
    <visual>
      <geometry><cylinder radius="0.025" length="0.3"/></geometry>
      <origin xyz="0 0 0.15" rpy="0 0 0"/>
      <material name="blue"><color rgba="0.3 0.3 0.8 1"/></material>
    </visual>
    <collision>
      <geometry><cylinder radius="0.025" length="0.3"/></geometry>
    </collision>
    <inertial>
      <mass value="1"/>
      <inertia ixx="0.01" ixy="0" ixz="0" iyy="0.01" iyz="0" izz="0.01"/>
    </inertial>
  </link>

  <!-- Camera link -->
  <joint name="camera_joint" type="fixed">
    <parent link="arm_link2"/>
    <child link="camera_link"/>
    <origin xyz="0 0 0.3" rpy="0 0 0"/>
  </joint>

  <link name="camera_link">
    <visual>
      <geometry><box size="0.1 0.05 0.05"/></geometry>
      <material name="green"><color rgba="0.3 0.8 0.3 1"/></material>
    </visual>
    <collision>
      <geometry><box size="0.1 0.05 0.05"/></geometry>
    </collision>
    <inertial>
      <mass value="0.5"/>
      <inertia ixx="0.01" ixy="0" ixz="0" iyy="0.01" iyz="0" izz="0.01"/>
    </inertial>
  </link>

  <!-- Transmissions -->
  <transmission name="trans_arm_joint1">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="arm_joint1">
      <hardwareInterface>PositionJointInterface</hardwareInterface>
    </joint>
    <actuator name="motor1">
      <hardwareInterface>PositionJointInterface</hardwareInterface>
      <mechanicalReduction>1</mechanicalReduction>
    </actuator>
  </transmission>

  <transmission name="trans_arm_joint2">
    <type>transmission_interface/SimpleTransmission</type>
    <joint name="arm_joint2">
      <hardwareInterface>PositionJointInterface</hardwareInterface>
    </joint>
    <actuator name="motor2">
      <hardwareInterface>PositionJointInterface</hardwareInterface>
      <mechanicalReduction>1</mechanicalReduction>
    </actuator>
  </transmission>

  <!-- Gazebo plugin -->
  <gazebo>
    <plugin name="gazebo_ros_control" filename="libgazebo_ros_control.so"/>
  </gazebo>

</robot>


my_tf_demo/config/controllers.yaml

In [ ]:
arm_joint1_position_controller:
  type: effort_controllers/JointPositionController
  joint: arm_joint1
  pid: {p: 100.0, i: 0.01, d: 1.0}

arm_joint2_position_controller:
  type: effort_controllers/JointPositionController
  joint: arm_joint2
  pid: {p: 100.0, i: 0.01, d: 1.0}


launch/arm_control.launch

In [ ]:
<launch>
  <!-- Gazebo -->
  <include file="$(find gazebo_ros)/launch/empty_world.launch"/>

  <!-- Robot description -->
  <param name="robot_description" command="$(find xacro)/xacro '$(find my_tf_demo)/urdf/robot_with_arm.urdf.xacro'"/>

  <!-- Spawn robot -->
  <node name="spawn_urdf" pkg="gazebo_ros" type="spawn_model"
        args="-param robot_description -urdf -model tf_demo_robot" />

  <!-- Controllers -->
  <rosparam file="$(find my_tf_demo)/config/controllers.yaml" command="load"/>
  <node name="controller_spawner" pkg="controller_manager" type="spawner"
        args="arm_joint1_position_controller arm_joint2_position_controller" />

  <!-- RViz -->
  <node name="rviz" pkg="rviz" type="rviz"/>
</launch>


In [ ]:
roslaunch my_tf_demo arm_control.launch

In [ ]:
rostopic pub /arm_joint1_position_controller/command std_msgs/Float64 "data: 0.5"
rostopic pub /arm_joint2_position_controller/command std_msgs/Float64 "data: -0.5"